## Playground

In [133]:
# https://biopython.org/wiki/GFF_Parsing

from typing import List, Dict, Iterable, Tuple
import pprint
import itertools as it
import collections

import Bio
from Bio.SeqFeature import SeqFeature, ExactPosition, BeforePosition, AfterPosition
from Bio.SeqFeature import FeatureLocation, CompoundLocation
from BCBio.GFF import GFFExaminer
from BCBio import GFF

# in_file = "../samples/prokka.gff3"
# in_file = "../samples/augustus.gff3"
in_file = "../samples/maker.gff3"


In [134]:
recs = list(GFF.parse(in_file))
pprint.pp(recs)

[SeqRecord(seq=UnknownSeq(1274862, character='?'), id='4', name='<unknown name>', description='<unknown description>', dbxrefs=[])]


In [135]:
entry = recs[0]
pprint.pp(entry)

SeqRecord(seq=UnknownSeq(1274862, character='?'), id='4', name='<unknown name>', description='<unknown description>', dbxrefs=[])


In [136]:
f = entry.features[0]
f

SeqFeature(FeatureLocation(ExactPosition(53460), ExactPosition(64200), strand=-1), type='gene', id='maker-4-est_gff_Cufflinks-gene-0.0')

In [137]:
f.qualifiers

{'ID': ['maker-4-est_gff_Cufflinks-gene-0.0'],
 'Name': ['maker-4-est_gff_Cufflinks-gene-0.0'],
 'score': ['7.864828'],
 'source': ['maker']}

## Formatting `SeqRecord` as DDBJ annotation table

In [138]:
len(recs)

1

In [139]:
collections.Counter(f.type for f in recs[0].features) 

Counter({'gene': 69})

In [140]:
a_gene = recs[0].features[0]
collections.Counter(f.type for f in a_gene.sub_features)

Counter({'mRNA': 1})

In [141]:
a_transcript = a_gene.sub_features[0]
collections.Counter(f.type for f in a_transcript.sub_features)

Counter({'exon': 5, 'five_prime_UTR': 1, 'CDS': 5, 'three_prime_UTR': 1})

In [142]:
a_transcript.sub_features

[SeqFeature(FeatureLocation(ExactPosition(63539), ExactPosition(64200), strand=-1), type='exon', id='maker-4-est_gff_Cufflinks-gene-0.0-mRNA-1:exon:4'),
 SeqFeature(FeatureLocation(ExactPosition(57141), ExactPosition(61911), strand=-1), type='exon', id='maker-4-est_gff_Cufflinks-gene-0.0-mRNA-1:exon:3'),
 SeqFeature(FeatureLocation(ExactPosition(56499), ExactPosition(57083), strand=-1), type='exon', id='maker-4-est_gff_Cufflinks-gene-0.0-mRNA-1:exon:2'),
 SeqFeature(FeatureLocation(ExactPosition(53816), ExactPosition(53999), strand=-1), type='exon', id='maker-4-est_gff_Cufflinks-gene-0.0-mRNA-1:exon:1'),
 SeqFeature(FeatureLocation(ExactPosition(53460), ExactPosition(53751), strand=-1), type='exon', id='maker-4-est_gff_Cufflinks-gene-0.0-mRNA-1:exon:0'),
 SeqFeature(FeatureLocation(ExactPosition(64050), ExactPosition(64200), strand=-1), type='five_prime_UTR', id='maker-4-est_gff_Cufflinks-gene-0.0-mRNA-1:five_prime_utr'),
 SeqFeature(FeatureLocation(ExactPosition(63539), ExactPosition(

In [143]:
target_type = "CDS"
target_features = [f for f in recs[0].features[0].sub_features[0].sub_features if f.type == target_type]
locs = [f.location for f in target_features]
compound_loc = CompoundLocation(locs)
feature_packed = SeqFeature(compound_loc, target_type)
feature_packed

SeqFeature(CompoundLocation([FeatureLocation(ExactPosition(63539), ExactPosition(64050), strand=-1), FeatureLocation(ExactPosition(57141), ExactPosition(61911), strand=-1), FeatureLocation(ExactPosition(56499), ExactPosition(57083), strand=-1), FeatureLocation(ExactPosition(53816), ExactPosition(53999), strand=-1), FeatureLocation(ExactPosition(53643), ExactPosition(53751), strand=-1)], 'join'), type='CDS', location_operator='join')

In [144]:
print(compound_loc)

join{[63539:64050](-), [57141:61911](-), [56499:57083](-), [53816:53999](-), [53643:53751](-)}


In [145]:
start_pos = AfterPosition(5)
end_pos = BeforePosition(10)
my_location = FeatureLocation(start_pos, end_pos)

In [146]:
str(my_location.start)

'>5'

In [147]:
target_features[2].qualifiers

{'ID': ['maker-4-est_gff_Cufflinks-gene-0.0-mRNA-1:cds'],
 'Parent': ['maker-4-est_gff_Cufflinks-gene-0.0-mRNA-1'],
 'source': ['maker'],
 'phase': ['2']}

In [148]:
x = compound_loc.parts[0]

In [149]:
type(x.start)

Bio.SeqFeature.ExactPosition

In [150]:
compound_loc.parts

[FeatureLocation(ExactPosition(63539), ExactPosition(64050), strand=-1),
 FeatureLocation(ExactPosition(57141), ExactPosition(61911), strand=-1),
 FeatureLocation(ExactPosition(56499), ExactPosition(57083), strand=-1),
 FeatureLocation(ExactPosition(53816), ExactPosition(53999), strand=-1),
 FeatureLocation(ExactPosition(53643), ExactPosition(53751), strand=-1)]

In [151]:
def table_to_str(table: List[List[str]]) -> str:
    return "\n".join("\t".join(items) for items in table)

In [152]:
def format_location(loc: Bio.SeqFeature.FeatureLocation) -> str:
    """Format Bio.SeqFeature.FeatureLocation
    """
    return "{start}..{end}".format(start=loc.start, end=loc.end)

In [162]:
def to_ddbj_table(rec: SeqRecord) -> List[List[str]]:
    """Convert GFF.SeqRecord into DDBJ annotation table format

    DDBJ annotation table format is TSV (tab-separated variables) with 5 columns
        - Column 1: Sequence ID
        - Column 2: Feature Key
        - Column 3: Location
        - Column 4: Qualifier Key
        - Column 5: Qualifier Value

    Many entries are blank.

    Example:
    [
        ["CLN01", "source", "1..12297"                         , "organism"   ,  "Mus musculus"  ],
        [       ,         ,                                    , "mol_type"   ,  "genomic DNA"   ],
        [       ,         ,                                    , "clone"      ,  "PC0110"        ],
        [       ,         ,                                    , "chromosome" ,  "8"             ],
        [       ,  "CDS"  , "join(<1..456,609..879,1070..1213)", "product"    ,  "protein kinase"],
        [       ,         ,                                    , "codon_start",  "2"             ],
    ]

    """
    lines = sum(
        len(vals) if isinstance(vals, list) else 1
        for feature in rec.features
        for vals in feature.qualifiers.values()
    )
    print("lines: {}".format(lines))
    table = [["" for _ in range(5)] for _ in range(lines)]
    table[0][0] = str(rec.id)

    idx = 0
    for feature in rec.features:
        is_first_line = True
        for (qualifier_key, values) in feature.qualifiers.items():
            values = values if isinstance(values, list) else [values]
            for qualifier_value in values:
                if is_first_line:
                    is_first_line = False
                    table[idx][1] = feature.type
                    if feature.location is not None:
                        table[idx][2] = format_location(feature.location)
                table[idx][3] = qualifier_key
                table[idx][4] = str(qualifier_value)
                idx += 1

    return table


In [163]:
tbl = to_ddbj_table(entry)
s = table_to_str(tbl)
print(s)

lines: 276
4	gene	53460..64200	ID	maker-4-est_gff_Cufflinks-gene-0.0
			Name	maker-4-est_gff_Cufflinks-gene-0.0
			score	7.864828
			source	maker
	gene	90013..128029	ID	maker-4-est_gff_Cufflinks-gene-1.0
			Name	maker-4-est_gff_Cufflinks-gene-1.0
			score	6.732889
			source	maker
	gene	152875..163632	ID	maker-4-est_gff_Cufflinks-gene-1.1
			Name	maker-4-est_gff_Cufflinks-gene-1.1
			score	16.569730
			source	maker
	gene	68503..76929	ID	maker-4-est_gff_Cufflinks-gene-1.2
			Name	maker-4-est_gff_Cufflinks-gene-1.2
			score	20.410109
			source	maker
	gene	86132..87863	ID	maker-4-est_gff_Cufflinks-gene-1.3
			Name	maker-4-est_gff_Cufflinks-gene-1.3
			score	2132.029560
			source	maker
	gene	135602..150357	ID	maker-4-est_gff_Cufflinks-gene-1.4
			Name	maker-4-est_gff_Cufflinks-gene-1.4
			score	5.745459
			source	maker
	gene	230940..235351	ID	maker-4-est_gff_Cufflinks-gene-2.0
			Name	maker-4-est_gff_Cufflinks-gene-2.0
			score	39.430468
			source	maker
	gene	248672..250746	ID	maker-4-est_g

In [18]:
def get_containment_relation(features: Iterable[SeqFeature]) -> Dict[SeqFeature, List[SeqFeature]]:
    """Find containment relation and return dictionary such that
    a key contains its value as sub-segments.
    """
    res = collections.defaultdict(list)
    for f1, f2 in it.combinations(features, 2):
        start1 = f1.location.start
        end1 = f1.location.end
        start2 = f2.location.start
        end2 = f2.location.end
        if start1 <= start2 and end2 <= end1:
            res[f1].append(f2)
        elif start2 <= start1 and end1 <= end2:
            res[f2].append(f1)
        elif end1 <= start2 or end2 <= start1:
            # note that biopython's segment is [start, end) 
            pass
        else:
            loc1 = str(f1.location)
            loc2 = str(f2.location)
            print("Intersection (0-based): {}:{} and {}:{}".format(f1.type, loc1, f2.type, loc2))
    return res

def group_by_id(features: Iterable[SeqFeature]) -> Dict[Tuple[str, str], List[SeqFeature]]:
    """Group segments by (ID, featureKey)
    """
    res = collections.defaultdict(list)
    for feature in features:
        tup = (feature.id, feature.type)
        res[tup].append(feature)
    return res

In [19]:
# Containment and intersection of segments do not work as different CDS can overlap anyway
get_containment_relation(entry.features)

Intersection (0-based): gene:[1208177:1213081](+) and gene:[1213069:1217101](-)


defaultdict(list, {})

In [20]:
# Grouping by (ID, featureKey)
d = group_by_id(entry.features)
for k, vs in d.items():
    print("{}: {}".format(k, len(vs)))

('maker-4-est_gff_Cufflinks-gene-0.0', 'gene'): 1
('maker-4-est_gff_Cufflinks-gene-1.0', 'gene'): 1
('maker-4-est_gff_Cufflinks-gene-1.1', 'gene'): 1
('maker-4-est_gff_Cufflinks-gene-1.2', 'gene'): 1
('maker-4-est_gff_Cufflinks-gene-1.3', 'gene'): 1
('maker-4-est_gff_Cufflinks-gene-1.4', 'gene'): 1
('maker-4-est_gff_Cufflinks-gene-2.0', 'gene'): 1
('maker-4-est_gff_Cufflinks-gene-2.1', 'gene'): 1
('maker-4-est_gff_Cufflinks-gene-2.2', 'gene'): 1
('maker-4-est_gff_Cufflinks-gene-2.3', 'gene'): 1
('maker-4-est_gff_Cufflinks-gene-2.4', 'gene'): 1
('maker-4-est_gff_Cufflinks-gene-2.5', 'gene'): 1
('maker-4-est_gff_Cufflinks-gene-2.6', 'gene'): 1
('maker-4-est_gff_Cufflinks-gene-2.7', 'gene'): 1
('maker-4-est_gff_Cufflinks-gene-2.8', 'gene'): 1
('maker-4-est_gff_Cufflinks-gene-2.9', 'gene'): 1
('maker-4-est_gff_Cufflinks-gene-3.0', 'gene'): 1
('maker-4-est_gff_Cufflinks-gene-3.1', 'gene'): 1
('maker-4-est_gff_Cufflinks-gene-3.2', 'gene'): 1
('maker-4-est_gff_Cufflinks-gene-3.3', 'gene'): 1


In [23]:
import collections
collections.Counter(f.type for f in entry.features[0].sub_features)

Counter({'mRNA': 1})

In [24]:
collections.Counter(f.type for f in entry.features[0].sub_features[0].sub_features)

Counter({'exon': 5, 'five_prime_UTR': 1, 'CDS': 5, 'three_prime_UTR': 1})

## Testing translators in `src`

In [26]:
import json
import pathlib
import sys

pwd = pathlib.Path.cwd().parent / "src"
sys.path.append(str(pwd))
from modules import translators

paths = ["../src/modules/gff_column_to_ddbj_qualifier.json", "../src/modules/gff_attribute_to_ddbj_qualifier.json"]
paths = [pathlib.Path(p).absolute() for p in paths]

In [27]:
qualifier_converter = translators.GFF3AttributesToQualifiers(paths)

In [28]:
entry = qualifier_converter.run(entry)

In [29]:
entry.features[-3].sub_features[0].qualifiers

{'note': ['source:maker'],
 '_AED': ['0.11'],
 '_eAED': ['0.11'],
 '_QI': ['30|1|1|1|0|0|5|103|441']}

In [30]:
cds = [f for f in entry.features[0].sub_features[0].sub_features if f.type == "CDS"]
cds[0].qualifiers

{'note': ['source:maker'], 'codon_start': ['0']}

In [31]:
collections.Counter(f.type for f in entry.features[0].sub_features[0].sub_features)

Counter({'exon': 5, 'five_prime_UTR': 1, 'CDS': 5, 'three_prime_UTR': 1})

In [32]:
paths = ["../src/modules/gff_type_to_ddbj_feature.json"]
paths = [pathlib.Path(p).absolute() for p in paths]
feature_converter = translators.GFF3TypesToFeatures(paths)

In [33]:
entry = feature_converter.run(entry)

In [34]:
collections.Counter(f.type for f in entry.features[0].sub_features[0].sub_features)

Counter({'exon': 5, "5'UTR": 1, 'CDS': 5, "3'UTR": 1})

In [35]:
type(entry.features)

list

## `assembly_gap`

In [14]:
from Bio.Seq import Seq
import Bio.SeqIO
from typing import List, Dict, Iterable, Tuple
import gzip

# https://www.ddbj.nig.ac.jp/news/ja/2021-08-10_3.html
fasta_path = "./fasta_na.BOUE01000001.txt.gz"

In [6]:
with gzip.open(fasta_path, 'rt') as f:
    seqs = Bio.SeqIO.parse(f, "fasta")
    for s in seqs:
        print(s.id)

BOUE01000001|BOUE01000001.1


In [17]:
s.seq.

AttributeError: 'Seq' object has no attribute 'seq'

In [ ]:
get

In [24]:
def get_assembly_gap(seq: Seq) -> List[Tuple[int, int]]:
    """
    Get assembly_gap locations from seq.

    [NOTE] A location is of format (begin, end)
    with 1-based indexing, and inclusive in both sides.

    >>> s = Seq("atatnnngattacanccc")
    [(5, 7), (15, 15)]
    """
    s = str(seq)
    patt = re.compile("n+")
    matches = patt.finditer(s)

    segments = []
    for m in matches:
        a, b = m.span()  # 0-based, left-inclusive, right-exclusive
        tup = (a + 1, b) # 1-based, both-inclusive
        segments.append(tup)

    return segments

In [29]:
s = Seq("atatnnngattacanccc")
get_assembly_gap(s)

[(5, 7), (15, 15)]

In [25]:
%timeit get_assembly_gap(s.seq)

267 ms ± 7.52 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
get_assembly_gap(s.seq)

[(848852, 848861),
 (1037995, 1038004),
 (1268390, 1268399),
 (1320750, 1320759),
 (1326129, 1326138),
 (1368923, 1368932),
 (1369648, 1369657),
 (1407014, 1407023),
 (1547450, 1547459),
 (1608547, 1608556),
 (1845750, 1845759),
 (2092435, 2092444),
 (2093497, 2093506),
 (2222806, 2222815),
 (2226754, 2226763),
 (2229737, 2229746),
 (2337459, 2337468),
 (2361686, 2361695),
 (2372429, 2372438),
 (2373972, 2373981),
 (2376826, 2378764),
 (2434780, 2434789),
 (2446911, 2446920),
 (2725397, 2725406),
 (2732641, 2732650),
 (2823806, 2823815),
 (2901099, 2901108),
 (2902164, 2902173),
 (2905544, 2905553),
 (2909241, 2909250),
 (2915487, 2915496),
 (2917500, 2917509),
 (3133148, 3133157),
 (3545811, 3545820),
 (3618053, 3618062),
 (3626493, 3626502),
 (3632244, 3632253),
 (3636722, 3636731),
 (3721270, 3721279),
 (3726246, 3726255),
 (4021210, 4021219),
 (4022548, 4022557),
 (4024195, 4024204),
 (4052792, 4052801),
 (4110186, 4110195),
 (4110492, 4110501),
 (4114367, 4114376),
 (4115237, 4115

In [8]:
import numpy as np

In [9]:
# Make it faster with numpy?
s_np = np.array(s.seq)
s_np.dtype

dtype('<U1')

In [48]:
indices = np.where(s_np == 'n')

In [ ]:
indices.

## YAML

In [104]:
from typing import Any, Dict, List, Tuple, Iterable, Union
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import SeqFeature
import yaml

path = "../samples/common.yaml"

In [105]:
with open(path, "r") as f:
    header_info = yaml.safe_load(f)

In [106]:
header_info

{'SUBMITTER': {'ab_name': ['Robertson,G.R.', 'Mishima,H.'],
  'contact': 'Hanako Mishima',
  'email': 'mishima@ddbj.nig.ac.jp',
  'phone': '81-55-981-6853',
  'institute': 'National Institute of Genetics',
  'country': 'Japan',
  'city': 'Mishima',
  'street': 'Yata 1111',
  'zip': '411-8540'},
 'REFERENCE': {'title': 'Mouse Genome Sequencing',
  'ab_name': ['Robertson,G.R.', 'Mishima,H'],
  'year': 2017,
  'status': 'Unpublished'},
 'COMMENT': {'line': ['Please visit our website URL',
   'http://www.ddbj.nig.ac.jp/']}}

In [164]:
def create_common(path) -> SeqFeature:
    """Create submitter feature

    """
    with open(path, "r") as f:
        header_info = yaml.safe_load(f)

    features = [SeqFeature(type=key, qualifiers=xs) for (key, xs) in header_info.items()]
    record = SeqRecord("", id="COMMON", features=features)
    return record



In [165]:
common = create_common(path)
common

SeqRecord(seq='', id='COMMON', name='<unknown name>', description='<unknown description>', dbxrefs=[])

In [166]:
s = table_to_str(to_ddbj_table(common))
print(s)

lines: 17
COMMON	SUBMITTER		ab_name	Robertson,G.R.
			ab_name	Mishima,H.
			contact	Hanako Mishima
			email	mishima@ddbj.nig.ac.jp
			phone	81-55-981-6853
			institute	National Institute of Genetics
			country	Japan
			city	Mishima
			street	Yata 1111
			zip	411-8540
	REFERENCE		title	Mouse Genome Sequencing
			ab_name	Robertson,G.R.
			ab_name	Mishima,H
			year	2017
			status	Unpublished
	COMMENT		line	Please visit our website URL
			line	http://www.ddbj.nig.ac.jp/


In [110]:
fs = common.features
fs

[SeqFeature(None, type='SUBMITTER'),
 SeqFeature(None, type='REFERENCE'),
 SeqFeature(None, type='COMMENT')]

In [115]:
fs[2].qualifiers

{'line': ['Please visit our website URL', 'http://www.ddbj.nig.ac.jp/']}

In [127]:
sss = "ddjflakdsj"
isinstance(sss, list)

False